In [ ]:
Nx=5
T=1.0
fpp=16

In [ ]:
import sympy
from sympy import *
    
x=Symbol('x')
y=Symbol('y')
t=Symbol('t')
lmd=1.0

In [ ]:
def f(t):
    return exp(-t)
def g(t):
    return exp(-lmd*t)

In [ ]:
# evaluates to 2.718281828 ...
#f(-1).evalf(10)

In [ ]:
def pdf1(x):
    """ pdf for failure times """
    return f(x)

def pdf2(y):
    """ pdf for censoring times """
    return g(y)

In [ ]:
def fa(t):
    """ conditional distribution given censoring time is bigger than failure time """
    return diff(integrate(integrate(pdf1(x)*pdf2(y), (x,y,oo)), (y,0,t)),t) 

def fb(t):
    """ conditional distribution given failure time is bigger than censoring time """
    return diff(integrate(integrate(pdf1(x)*pdf2(y), (y,x,oo)), (x,0,t)),t) 

def fab(t): 
    """ conditional distribution given both failure and censoring times exceed t """
    return fa(t) + fb(t)

In [ ]:
Gshat=1
sumKME=0
KM_Efron=0

In [ ]:
# Case 1: i = 0, X_k, Y_k > t
def cdf1(x):
    return integrate(pdf1(t), (t, 0, x))
def cdf2(y):
    return integrate(pdf2(t), (t, 0, y)) 

In [ ]:
Pr = (1 - cdf1(t))**(Nx) * (1 - cdf2(t))**(Nx)
sumKME = sumKME + Gshat * Pr
print("i ", "j ", "bj ", "Gshat ", "Pr ", "incKME ", "sumKME ")
print(0,0,bin(0)[2:].zfill(Nx),Gshat, Pr.evalf(fpp, subs={t:T}), (Gshat*Pr).evalf(fpp,subs={t:T}), sumKME.evalf(fpp, subs={t:T}))

In [ ]:
for i in range(Nx+1)[1:]:
    incKME=0	
    for j in range(2**i):
        bj=bin(j)[2:]
        lbj=len(bj)
        if j==0: lbj=1 
        p=1
        Gshat=1
		
        for k1 in range(i-lbj+1)[1:]:
            p=integrate(p*fa(str(t)+str(k1)),(str(t)+str(k1),0,str(t)+str(k1+1)))

        for k2 in range(i+1)[(i-lbj+1):]:
            d=('q'+bj)[k2-i+lbj]
            if d=='0':
                fs=fa(str(t)+str(k2))
            else:
                fs=fb(str(t)+str(k2))
                Gshat=Gshat*((Nx-k2)/(Nx-k2+1))
            
            if k2<i:
                p=integrate(p*fs,(str(t)+str(k2),0,str(t)+str(k2+1)))
            else:
                p=integrate(p*fs,(str(t)+str(i),0,t))
            

        for k3 in range(Nx+1)[i+1:]:
            if k3<Nx:
                p=integrate(p*fab(str(t)+str(k3)), (str(t)+str(k3), t, str(t)+str(k3+1)))
            else:
                p=integrate(p*fab(str(t)+str(Nx)),(str(t)+str(Nx),t,oo))

        Pr = factorial(Nx) * p
        incKME = Pr*Gshat
        sumKME = sumKME + incKME
        print(i,", ", j, ", ", bj, ", ", Gshat)#, ", ", Pr.evalf(fpp, subs={t:T}), ", ", (Gshat*Pr).evalf(fpp,subs={t:T}), \
              #", ", sumKME.evalf(fpp, subs={t:T}))
    
	
	# Store the KM-Efron after i=Nx-1 
    if i==Nx-1: 
        KM_Efron = sumKME
        print("\nKM-Efron = ", sumKME.evalf(fpp, subs={t:T}),"\n")

In [ ]:
print("For N = {}, at T = {}: ".format(Nx, T))
print("S(T) = {} ".format(f(T).evalf(fpp)))
print("KM-Efron = {} ".format(KM_Efron.evalf(fpp, subs={t:T})))
print("KM-Gill = {} ".format(sumKME.evalf(fpp, subs={t:T})))
print("Bias (Efron) = KM-Efron - S(T) = {:.4}".format((KM_Efron - f(T)).evalf(fpp, subs={t:T})))
print("Bias (Gill) = KM-Gill - S(T) = {:.4}".format((sumKME - f(T)).evalf(fpp, subs={t:T})))